In [1]:
import os

In [2]:
%pwd

'c:\\Users\\artit\\IKP_2025\\fraud_prediction\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\artit\\IKP_2025\\fraud_prediction'

In [5]:
from pathlib import Path
import sys

project_path = r"c:\Users\artit\IKP_2025\fraud_prediction"
os.chdir(project_path)

if os.path.abspath("src") not in sys.path:
    sys.path.append(os.path.abspath("src"))

print(f"Path now : {os.getcwd()}")
print(f" Config path : {Path('config/config.yaml').exists()}")

Path now : c:\Users\artit\IKP_2025\fraud_prediction
 Config path : True


In [6]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    onedrive_file_path: str
    local_data_file: Path
    unzip_dir: Path

In [7]:
from fraud_prediction.constants import *
from fraud_prediction.utils.common import read_yaml, create_directories

In [8]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            onedrive_file_path=config.onedrive_file_path,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config
      

In [9]:
import os
import zipfile
import gdown
from fraud_prediction import logger
from fraud_prediction.utils.common import get_size
from fraud_prediction.config.configuration import ConfigurationManager

In [10]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

     
    def download_file(self) -> str:
        try: 
            #dataset_url = self.config.onedrive_file_path
            zip_download_dir = self.config.local_data_file
            
            # Create destination directory 
            os.makedirs(os.path.dirname(zip_download_dir), exist_ok=True)
            file_id = "1xHzaCHq6gEXFmYN2Cc2Yb1jVxaAayb2y"

            if os.path.exists(self.config.local_data_file):
                os.remove(self.config.local_data_file)
                logger.info(f"Removed corrupted file : {zip_download_dir}")

            logger.info(f"Downloading data from : {file_id}")

            gdown.download(id=file_id, output=str(zip_download_dir), quiet=False)

            logger.info(f"Downloaded successfully into {zip_download_dir}")

        except Exception as e:
            raise e
        
    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)




In [13]:
try: 
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)

    #skip download for now
    data_ingestion.extract_zip_file() 

    print("Data extracted successfully")
except Exception as e:
    print(f"Error occurred : {e}")

[2026-01-22 11:39:43,543: INFO: common: yaml file: config\config.yaml loaded successfully]
[2026-01-22 11:39:43,544: INFO: common: yaml file: params.yaml loaded successfully]
[2026-01-22 11:39:43,550: INFO: common: created directory at: artifacts]
[2026-01-22 11:39:43,552: INFO: common: created directory at: artifacts/data_ingestion]


Data extracted successfully


In [ ]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e